In [13]:
import torch, torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
import PIL
from PIL import Image
import matplotlib.pyplot as plt


In [25]:
def show(tensor, num=25, wandbactive=0, name=''):
    data = tensor.detach().cpu()
    grid = make_grid(data[:num], nrow=5).permute(1,2,0)
    
    if  wandbactive==1 :
        wandb.log({name:wandb.Image(grid.numpy().clip(0,1))})
        
    plt.imshow(grid.clip(0,1))
    plt.show()

In [3]:
n_epochs = 500
batch_size = 128
lr = 1e-4
z_dim = 256
device = "cuda"

cur_step = 0
crit_cycles = 5
gen_losses = []
critic_losses = []
show_steps = 50
save_steps = 100

wandbact = 1

In [4]:
import wandb
wandb.login(key="95eda70b703e651a93829d62187e30b7bc107306")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sparrowblack437 (peakysparrow). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Guhan/.netrc


True

In [5]:
%%capture
exp_name = wandb.util.generate_id()

myrun = wandb.init(
    project = "wGAN",
    group = "exp_name",
    config = {
        "optimizer" : "sgd",
        "model" : "wgan_gp",
        "epoch" : "1000",
        "batch_size" : 128
        
    }
)

config = wandb.config


In [6]:
print(exp_name)

4c1z4v63


In [8]:
class Generator(nn.Module):
    def __init__(self, z_dim=64, d_dim=16):
        super(Generator, self).__init__()
        
        self.z_dim = z_dim
        
        self.gen = nn.Sequential(
            ###img_shape = n-1 * stride - 2*padding + kernel_size
            ###init_img_shape = 1 x 1
            nn.ConvTranspose2d(z_dim, d_dim*32, 4, 1, 0), # = 4x4 image with ch 256 => 512
            nn.BatchNorm2d(d_dim * 32),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(d_dim*32, d_dim*16, 4, 2, 1), # = 8x8 image with ch 512 => 256
            nn.BatchNorm2d(d_dim * 16),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(d_dim*16, d_dim*8, 4, 2, 1), # = 16x16 image with ch 256 => 128
            nn.BatchNorm2d(d_dim * 8),
            nn.ReLU(True),            
                        
            nn.ConvTranspose2d(d_dim*8, d_dim*4, 4, 2, 1), # = 32x32 image with ch 128 => 64
            nn.BatchNorm2d(d_dim * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(d_dim*4, d_dim*2, 4, 2, 1), # = 64x64 image with ch 64 => 32
            nn.BatchNorm2d(d_dim * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(d_dim*2, 3, 4, 2, 1), # = 128x128 image with ch 32 => 3
            nn.Tanh() #  produce result in the range -1 to 1     
                             
        ) 
        
        def forward(self, noise):
            x = noise.view(len(noise), self.z_dim, 1, 1)
            return self.gen(x)

def gen_noise(num, z_dim, device="cuda"):
    return torch.randn(num, z_dim, device=device)

In [9]:
class Critic(nn.Module):
    def __init__(self, d_dim=16):
        super(Critic, self).__init__()
        
        self.critic = nn.Sequential(
            # img_shape = (n+2*pad-ker_size) // stride+1
            # init_img_shape = 128x128
            nn.Conv2d(3, d_dim, 4, 2, 1),  # = 64x64 image with ch 3 => 16
            nn.InstanceNorm2d(d_dim),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(d_dim, d_dim*2, 4, 2, 1),  # = 32x32 image with ch 16 => 32
            nn.InstanceNorm2d(d_dim*2),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(d_dim*2, d_dim*4, 4, 2, 1),  # = 16x16 image with ch 32 => 64
            nn.InstanceNorm2d(d_dim*4),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(d_dim*4, d_dim*8, 4, 2, 1),  # = 8x8 image with ch 64 => 128
            nn.InstanceNorm2d(d_dim*8),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(d_dim*8, d_dim*16, 4, 2, 1),  # = 4x4 image with ch 128 => 256
            nn.InstanceNorm2d(d_dim*16),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(d_dim*16, 1, 4, 1, 0),  # = 1x1 image with ch 256 => 1
            
        )
        
    def forward(self, image):
        # image : 128 x 3 x 128 x 128
        critic_pred = self.critic(image) # == 128 x 1 x 1 x 1 
        return critic_pred.view(len(critic_pred), -1) # 128 x 1

In [11]:
## intit weights for layers 

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
        
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0) 
        
#gen = gen.apply(init_weights)
#critic = critic.apply(inint_weights)    
   

In [ ]:
# Load Dataset
import gdown, zipfile

url = "https://drive.google.com/drive/folders/0B7EVK8r0v71pTUZsaXdaSnZBZzg?resourcekey=0-rJlzl934LzC-Xp28GeIBzQ"
path = 'G:/Datasets/celeba'
download_path = path+'/img_align_celeba.zip'

if not os.path.exists(path):
    os.makedirs(path)

gdown.download(url, download_path, quiet = False)

with zipfile.ZipFile(download_path, 'r') as Ziphandler:
    Ziphandler.extractall(path)    



In [14]:
class Dataset(Dataset):
    def __init__(self, path, size=128, lim=1000):
        self.sizes = [size, size]
        items, labels = [], []
        
        for data in os.listdir(path)[:lim]:
            item = os.path.join(path, data)
            items.append(item)
            labels.append(data)
        self.items = items
        self.labes = labels
        
        def __len__(self):
            return len(self.items)
        
        def __getitem__(self, idx):
            data = PIL.Image.open(self.items[idx]).convert('RGB') # (width, height)
            data = np.asarray(torchvision.transforms.Resize(self.sizes)(data)) # 128 x 128 x 3
            data = np.transpose(data, (2, 0, 1)).astype(np.float32, copy=False) # 3 x 128 x 128
            data = torch.from_numpy(data).div(255) # from 0 to 1
            return data, self.labels[idx]

In [ ]:
## Data

data_path = download_path = path+'/img_align_celeba'
ds = Dataset(data_path, size=128, lim=10000)

## DataLoader

dataloader = DataLoader(ds, batch_size=batch_size, shuffle=True)

## Models

gen = Generator(z_dim).to(device)
critic = Critic().to(device)

## Optimizer

gen_opt = torch.optim.Adam(gen.parameters(), lr = lr, betas=(0.5, 0.9))
critic_opt = torch.optim.Adam(critic.parameters(), lr = lr, betas=(0.5, 0.9))

## Initialisations

#gen = gen.apply(init_weights)
#critic = critic.apply(inint_weights) 

# Wandb

if wandbact == 1:
    wandb.watch(gen, log_freq=100)
    wandb.watch(critic, log_freq=100)

In [ ]:
## test

x, y = next(iter(dataloader))
show(x)
print(y)

In [17]:
## gradient penalty calaculation

def get_gp(real, fake, critic, alpha, gamma=10):
    mix_images = real * alpha + fake * (1 - alpha) # 128 x 1 x 128 x 128 , linear interpolation
    mix_scores = critic(mix_images) # 128 x 1
    
    gradient = torch.autograd.grad(
        inputs = mix_images,
        outputs = mix_scores,
        grad_outputs = torch.ones_like(mix_scores),
        retain_graph = True,
        create_graph = True
    )[0]  # 128 x 3 x 128 x 128
    
    gradient = gradient.view(len(gradient), -1) # 128 x 3*128*128
    gradient_norm = gradient.norm(2, dim = 1)
    gp = gamma * ((gradient_norm - 1)**2).mean()
    
    return gp

In [22]:
## save checkpoints 

save_path = "G:/Software/"

def save_checkpoint(name):
    
    ## Gen model
    torch.save({
        'epoch' : epoch, 
        'model_state_dict' :gen.state_dict(),
        'optimizer_state_dict' : gen_opt.state_dict()
    }, save_path+'G-'+name+'.pkl')
    
    
    ## Critic Model
    torch.save({
        'epoch' : epoch, 
        'model_state_dict' :critic.state_dict(),
        'optimizer_state_dict' : critic_opt.state_dict()
    }, save_path+'C-'+name+'.pkl')
    
    print("Saved Checkpoint")
    

In [23]:
## Load Checkpoint

def load_checkpoint(name):
    
    ## Load Gen
    chkpt = torch.load(save_path+'G-'+name+'.pkl')
    gen.load_state_dict(chkpt['model_state_dict'])
    gen_opt.load_state_dict(chkpt['optimizer_state_dict'])
    
    ## Load Critic
    chkpt = torch.load(save_path+'C-'+name+'.pkl')
    critic.load_state_dict(chkpt['model_state_dict'])
    critic_opt.load_state_dict(chkpt['optimizer_state_dict'])
    
    print('Loaded Checkpoint')

In [ ]:
# Test save and load

epoch = 1
save_checkpoint("test")

load_checkpoint("test")

In [ ]:
for epoch in range(n_epochs):
    for real, _ in tqdm(dataloader):
        cur_bs = len(real) # 128
        real = real.to(device)
        
        ## Critic
        mean_critic_loss = 0
        
        for _ in range(crit_cycles):
            critic_opt.zero_grad()
            
            noise = gen_noise(cur_bs, z_dim)
            fake = gen(noise)
            critic_fake_pred = critic(fake.detach())
            critic_real_pred = critic(real)
            
            alpha = torch.rand(len(real), 1, 1, 1, device=device, requires_grad=True) # 128 x 1 x 1 x 1
            gp = get_gp(real, fake.detach(), critic, alpha)
            
            critic_loss = critic_fake_pred.mean() - critic_real_pred.mean() + gp 
            
            mean_critic_loss += critic_loss.item() / crit_cycles
            
            critic_loss.backward(retain_graph = True)
            critic_opt.step()
            
        critic_losses += [mean_critic_loss]
        
        ## Generator
        
        gen_opt.zero_grad()
        noise = gen_noise(cur_bs, z_dim)
        fake = gen(noise)
        critic_fake_pred = critic(fake)
        
        gen_loss = critic_fake_pred.mean()
        gen_loss.backward()
        gen_opt.step()
        
        gen_losses += [gen_loss.item()]
        
        ### Stats
        
        if wandbact == 1:
            wandb.log('Epoch =', epoch, 'Step =', cur_step, "Critic_Loss =", mean_critic_loss, "Gen_Loss =", gen_loss )
            
            
        if cur_step % save_steps == 0 and cur_step > 0:
            print("Saving Checkpoint:", cur_step, save_steps)
            save_checkpoint("wGAN-CelebFace")
            
        if cur_step % show_steps == 0 and cur_step > 0:
            show(fake, wandbactive=1 , name='fake')
            show(real, wandbactive=1 , name='real')
            
            gen_mean = sum(gen_losses[-show_steps:]) / show_steps
            critic_mean = sum(critic_losses[-show_steps:]) / show_steps
            print('Epoch =', epoch, 'Step =', cur_step, "Critic_Loss =", critic_mean, "Gen_Loss =", gen_mean )
            
            plt.plot(
                range(len(gen_losses)),
                torch.Tensor(gen_losses),
                label="Generator Loss"
            )
            
            plt.plot(
                range(len(gen_losses)),
                torch.Tensor(critic_losses),
                label="Critic Loss"
            )
            
            plt.ylim(-1000, 1000)
            plt.legend()
            plt.show()   
            
        cur_step += 1 

In [ ]:
## Generate faces

noise = gen_noise(batch_size, z_dim)
fake = gen(noise)
show(fake)

In [ ]:
plt.imshow(fake[4].detach().cpu().permute(1, 2, 0).squeeze().clip(0, 1))

In [ ]:
# morphing with interpolation between random points in latent space

from mpl_toolkits.axes_grid1 import ImageGrid

gen_set = []
z_shape = [1 , z_dim, 1, 1]
rows = 4
steps = 20

for i in range(rows):
    z1, z2 = torch.randn(z_shape), torch.randn(z_shape)
    for alpha in np.linspace(0, 1, steps):
        z = alpha*(z1) + (1 - alpha) * z2
        res = gen(z.cuda())[0]
        gen_set.append(res)

fig = plt.figure(figsize=(25, 11))
grid = ImageGrid(fig, 111, nrows_ncols = (rows, steps), axes_pad = 0.1)

for ax, img in grid(gen_set):
    ax.axis('off')
    res = img.cpu().detach().permute(1, 2, 0)
    res =  res - res.min()
    res = res/res.max() - res.min() 
    ax.imshow(res)
plt.show()
        